In [102]:
import torch 
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import time



In [103]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data = torchvision.datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.CIFAR10('data', train=False, download=True, transform=transform)

In [104]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, pin_memory=True, num_workers=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, pin_memory=True, num_workers=4, shuffle=False)

In [105]:
def conv_blk(inputChanneles, outputChannels, pooling):
    layers = [nn.Conv2d(inputChanneles, outputChannels, kernel_size=3, padding=1), nn.BatchNorm2d(outputChannels), nn.ReLU()]

    if pooling:
        layers.append(nn.MaxPool2d(2))

    return nn.Sequential(*layers)
    

In [106]:
class Network(nn.Module):
    def __init__(self, inputChannels, outputChannels):
        super(Network, self).__init__()
        self._block = nn.Sequential(
            conv_blk(inputChannels, 64, False),
            conv_blk(64, 64, False),
            conv_blk(64, 128, True),
            conv_blk(128, 128, False),
            conv_blk(128, 256, True),
            conv_blk(256, 256, False),
            conv_blk(256, 512, True),
            conv_blk(512, 512, False)
        )

        self._adaptive_pool = nn.AdaptiveAvgPool2d((1,1))

        self._res = nn.Linear(512, outputChannels)
    
    def forward(self, X):
        out = self._block(X)
        out = self._adaptive_pool(out)
        out =  torch.flatten(out, 1)
        out = self._res(out)
        return out


In [107]:
def train_once(model, train_loader, criterion, optimizer, device):
    avg_accuracy = []
    avg_loss = []

    for data in train_loader:
        image = data[0].to(device)    
        label = data[1].to(device)    

        optimizer.zero_grad()

        mod = model(image)

        loss = criterion(mod, label)
        avg_loss.append(loss.item())

        loss.backward()
        optimizer.step()

        avg_accuracy.append((torch.argmax(mod, dim=1) == label).float().mean())

    return torch.tensor(avg_loss).mean().cpu().detach().numpy(), torch.tensor(avg_accuracy).mean().cpu().detach().numpy()


In [ ]:
def __main__():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    model = Network(3, 10).to(device)

    learning_rate = 0.01

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr = learning_rate)

    loss_list = []
    acc_list = []

    epochs = 20
    print("Start:")
    for epoch in range(1, epochs + 1):

        epoch_loss, train_acc = train_once(model, train_loader, criterion, optimizer, device)

        print(f"Epoch {epoch:02d}/{epochs} | loss={epoch_loss:.4f} | "f"train_acc≈{train_acc:.4f}")

        loss_list.append(epoch_loss)
        acc_list.append(train_acc)


    loss, accuracy, cur_total = 0, 0, 0

    with torch.no_grad():
        for data in test_loader:
            images = data[0].to(device)
            labels = data[1].to(device)

            u = model(images)

            loss += criterion(u, labels).item()
            cur_total += labels.size(0)

            _, predicted_labels = torch.max(u.data, 1)

            accuracy += (predicted_labels == labels).sum().item()
        
    print(f"Loss: {loss / cur_total:.4f}, Test Accuracy: {accuracy / cur_total:.4f}")  


In [ ]:
if __name__ == "__main__":
  __main__()



cuda
Start:
Epoch 01/20 | loss=1.5886 | train_acc≈0.4148
Epoch 02/20 | loss=0.9589 | train_acc≈0.6596
Epoch 03/20 | loss=0.6716 | train_acc≈0.7649
Epoch 04/20 | loss=0.5140 | train_acc≈0.8219
Epoch 05/20 | loss=0.3991 | train_acc≈0.8622
Epoch 06/20 | loss=0.3032 | train_acc≈0.8955
Epoch 07/20 | loss=0.2324 | train_acc≈0.9184
Epoch 08/20 | loss=0.1641 | train_acc≈0.9414
Epoch 09/20 | loss=0.1225 | train_acc≈0.9569
Epoch 10/20 | loss=0.0939 | train_acc≈0.9668
Epoch 11/20 | loss=0.0783 | train_acc≈0.9724
Epoch 12/20 | loss=0.0684 | train_acc≈0.9758
Epoch 13/20 | loss=0.0634 | train_acc≈0.9782
Epoch 14/20 | loss=0.0542 | train_acc≈0.9817
Epoch 15/20 | loss=0.0503 | train_acc≈0.9829
Epoch 16/20 | loss=0.0461 | train_acc≈0.9842
Epoch 17/20 | loss=0.0484 | train_acc≈0.9845
Epoch 18/20 | loss=0.0367 | train_acc≈0.9875
Epoch 19/20 | loss=0.0410 | train_acc≈0.9862
Epoch 20/20 | loss=0.0368 | train_acc≈0.9876
Loss: 0.0113, Test Accuracy: 0.8585
